<a href="https://colab.research.google.com/github/majetikalyan007/ML-algorithms/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import log2
from collections import Counter, defaultdict

class Node:
    def __init__(self, attribute=None, threshold=None, is_leaf=False, prediction=None):
        # attribute: attribute name to split on (None for leaf)
        # threshold: for numeric attributes, the split threshold (<= goes left)
        self.attribute = attribute
        self.threshold = threshold
        self.children = {}  # for categorical: value -> Node; for numeric: 'le' and 'gt'
        self.is_leaf = is_leaf
        self.prediction = prediction

    def add_child(self, key, node):
        self.children[key] = node

    def __repr__(self):
        if self.is_leaf:
            return f"Leaf({self.prediction})"
        if self.threshold is None:
            return f"Node({self.attribute})"
        return f"Node({self.attribute} <= {self.threshold})"

# --- Utility functions ---

def entropy(labels):
    total = len(labels)
    if total == 0:
        return 0
    counts = Counter(labels)
    ent = 0.0
    for c in counts.values():
        p = c / total
        ent -= p * log2(p)
    return ent

def partition(dataset, attribute):
    """Return mapping from attribute value (or 'le'/'gt' for numeric) to subset of dataset rows."""
    # detect numeric vs categorical by checking the type of the first non-none value
    for row in dataset:
        if attribute in row and row[attribute] is not None:
            first_val = row[attribute]
            break
    else:
        # attribute missing in all rows
        return {}

    if isinstance(first_val, (int, float)):
        # numeric: split on median
        vals = [row[attribute] for row in dataset if attribute in row]
        thresh = sorted(vals)[len(vals)//2]
        parts = {'le': [], 'gt': []}
        for row in dataset:
            if attribute not in row or row[attribute] is None:
                # treat missing as separate bucket
                parts.setdefault('missing', []).append(row)
            elif row[attribute] <= thresh:
                parts['le'].append(row)
            else:
                parts['gt'].append(row)
        return parts, thresh
    else:
        parts = defaultdict(list)
        for row in dataset:
            val = row.get(attribute, 'missing')
            parts[val].append(row)
        return dict(parts), None


In [ ]:
def information_gain(dataset, attribute, target_attribute):
    base_entropy = entropy([row[target_attribute] for row in dataset])
    partitioned = partition(dataset, attribute)
    # partition returns (parts, thresh) for numeric, or (parts, None) for categorical
    if isinstance(partitioned, tuple):
        parts, _ = partitioned
    else:
        parts = partitioned
    total = len(dataset)
    remainder = 0.0
    for subset in parts.values():
        if len(subset) == 0:
            continue
        weight = len(subset) / total
        remainder += weight * entropy([r[target_attribute] for r in subset])
    return base_entropy - remainder

def majority_label(dataset, target_attribute):
    cnt = Counter([row[target_attribute] for row in dataset])
    return cnt.most_common(1)[0][0]

# --- ID3 recursive algorithm ---

def id3(dataset, attributes, target_attribute, depth=0, max_depth=None, min_samples=1):
    # If all examples have same label -> leaf
    labels = [row[target_attribute] for row in dataset]
    if len(set(labels)) == 1:
        return Node(is_leaf=True, prediction=labels[0])

    # If no attributes left or other stopping conditions
    if not attributes or (max_depth is not None and depth >= max_depth) or len(dataset) <= min_samples:
        return Node(is_leaf=True, prediction=majority_label(dataset, target_attribute))

    # Choose the best attribute by information gain
    best_attr = None
    best_gain = -1
    best_thresh = None
    for attr in attributes:
        parts_thresh = partition(dataset, attr)
        if not parts_thresh:
            continue
        if isinstance(parts_thresh, tuple):
            parts, thresh = parts_thresh
        else:
            parts, thresh = parts_thresh, None
        gain = information_gain(dataset, attr, target_attribute)
        if gain > best_gain:
            best_gain = gain
            best_attr = attr
            best_thresh = thresh

    if best_attr is None:
        return Node(is_leaf=True, prediction=majority_label(dataset, target_attribute))

    node = Node(attribute=best_attr, threshold=best_thresh)

    parts, _ = partition(dataset, best_attr) if isinstance(partition(dataset, best_attr), tuple) else (partition(dataset, best_attr), None)

    # Remove chosen attribute for subsequent splits if it's categorical
    remaining_attrs = [a for a in attributes if a != best_attr]

    for key, subset in parts.items():
        if not subset:
            node.add_child(key, Node(is_leaf=True, prediction=majority_label(dataset, target_attribute)))
        else:
            # For numeric, keep attribute in attributes list (we can split again on numeric with new threshold)
            child = id3(subset, remaining_attrs, target_attribute, depth+1, max_depth, min_samples)
            node.add_child(key, child)

    return node



In [ ]:
def predict(node, row):
    while not node.is_leaf:
        attr = node.attribute
        if node.threshold is not None:
            # numeric split
            val = row.get(attr, None)
            if val is None:
                # missing: try 'missing' child, else majority branch
                if 'missing' in node.children:
                    node = node.children['missing']
                    continue
                # fallthrough: pick larger child
                node = node.children.get('le') or node.children.get('gt')
                continue
            branch = 'le' if val <= node.threshold else 'gt'
            node = node.children.get(branch, Node(is_leaf=True, prediction=None))
        else:
            val = row.get(attr, 'missing')
            node = node.children.get(val)
            if node is None:
                # unseen attribute value: can't traverse; return None
                return None
    return node.prediction

# --- Pretty-print tree ---

def print_tree(node, indent=""):
    if node.is_leaf:
        print(indent + "-> Predict:", node.prediction)
        return
    if node.threshold is not None:
        print(indent + f"[{node.attribute} <= {node.threshold}]")
        if 'le' in node.children:
            print(indent + "  (<=) ")
            print_tree(node.children['le'], indent + "    ")
        if 'gt' in node.children:
            print(indent + "  (>) ")
            print_tree(node.children['gt'], indent + "    ")
        if 'missing' in node.children:
            print(indent + "  (missing) ")
            print_tree(node.children['missing'], indent + "    ")
    else:
        print(indent + f"[{node.attribute}]")
        for val, child in node.children.items():
            print(indent + f"  (== {val})")
            print_tree(child, indent + "    ")

# --- Example usage ---

def main():
    # Example dataset: simplified 'Play Tennis' (categorical + numeric example)
    # Each row is a dict of attributes; target label 'Play'
    dataset = [
        {'Outlook': 'Sunny', 'Temperature': 85, 'Humidity': 85, 'Windy': False, 'Play': 'No'},
        {'Outlook': 'Sunny', 'Temperature': 80, 'Humidity': 90, 'Windy': True, 'Play': 'No'},
        {'Outlook': 'Overcast', 'Temperature': 83, 'Humidity': 78, 'Windy': False, 'Play': 'Yes'},
        {'Outlook': 'Rain', 'Temperature': 70, 'Humidity': 96, 'Windy': False, 'Play': 'Yes'},
        {'Outlook': 'Rain', 'Temperature': 68, 'Humidity': 80, 'Windy': False, 'Play': 'Yes'},
        {'Outlook': 'Rain', 'Temperature': 65, 'Humidity': 70, 'Windy': True, 'Play': 'No'},
        {'Outlook': 'Overcast', 'Temperature': 64, 'Humidity': 65, 'Windy': True, 'Play': 'Yes'},
        {'Outlook': 'Sunny', 'Temperature': 72, 'Humidity': 95, 'Windy': False, 'Play': 'No'},
        {'Outlook': 'Sunny', 'Temperature': 69, 'Humidity': 70, 'Windy': False, 'Play': 'Yes'},
        {'Outlook': 'Rain', 'Temperature': 75, 'Humidity': 80, 'Windy': False, 'Play': 'Yes'},
        {'Outlook': 'Sunny', 'Temperature': 75, 'Humidity': 70, 'Windy': True, 'Play': 'Yes'},
        {'Outlook': 'Overcast', 'Temperature': 72, 'Humidity': 90, 'Windy': True, 'Play': 'Yes'},
        {'Outlook': 'Overcast', 'Temperature': 81, 'Humidity': 75, 'Windy': False, 'Play': 'Yes'},
        {'Outlook': 'Rain', 'Temperature': 71, 'Humidity': 80, 'Windy': True, 'Play': 'No'},
    ]

    attributes = ['Outlook', 'Temperature', 'Humidity', 'Windy']
    target = 'Play'

    tree = id3(dataset, attributes, target)
    print("Constructed decision tree:\n")
    print_tree(tree)

    # Test prediction
    test = {'Outlook': 'Sunny', 'Temperature': 66, 'Humidity': 90, 'Windy': True}
    pred = predict(tree, test)
    print('\nTest row:', test)
    print('Predicted label:', pred)

if __name__ == '__main__':
    main()


Constructed decision tree:

[Outlook]
  (== Sunny)
    [Temperature <= 75]
      (<=) 
        [Humidity <= 70]
          (<=) 
            -> Predict: Yes
          (>) 
            -> Predict: No
      (>) 
        -> Predict: No
  (== Overcast)
    -> Predict: Yes
  (== Rain)
    [Windy <= False]
      (<=) 
        -> Predict: Yes
      (>) 
        -> Predict: No

Test row: {'Outlook': 'Sunny', 'Temperature': 66, 'Humidity': 90, 'Windy': True}
Predicted label: No
